In [ ]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')

df = df.drop(columns=["id"], errors="ignore")

print(df.head())

df_limpo = df[
    (df["age"] >= 10000) & (df["age"] <= 30000) &
    (df["height"] >= 120) & (df["height"] <= 220) &
    (df["weight"] >= 30) & (df["weight"] <= 200) &
    (df["ap_hi"] >= 90) & (df["ap_hi"] <= 250) &
    (df["ap_lo"] >= 60) & (df["ap_lo"] <= 150)
].copy()

df_limpo["age_years"] = (df_limpo["age"] / 365.25).astype(int)

X = df_limpo.drop(columns=["cardio"])
y = df_limpo["cardio"]

provavel_categoricas = ["gender", "cholesterol", "gluc", "smoke", "alco", "active"]

features_all = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

features_bin = [col for col in features_all if col in provavel_categoricas]
features_cont = [col for col in features_all if col not in provavel_categoricas]

print(", ".join(features_cont))
print(", ".join(features_bin))

print("age_years   -> Idade em anos (convertida de 'age' em dias)")
print("height      -> Altura em centímetros")
print("weight      -> Peso em quilogramas")
print("ap_hi       -> Pressão arterial sistólica (máxima)")
print("ap_lo       -> Pressão arterial diastólica (mínima)")

print("gender      -> Sexo: 1 = homem, 2 = mulher")
print("cholesterol -> Nível de colesterol: 1 = normal, 2 = alto, 3 = muito alto")
print("gluc        -> Nível de glicose: 1 = normal, 2 = alto, 3 = muito alto")
print("smoke       -> Fumante: 0 = não, 1 = sim")
print("alco        -> Consome álcool: 0 = não, 1 = sim")
print("active      -> Fisicamente ativo: 0 = não, 1 = sim")

print("cardio      -> Presença de doença cardiovascular: 0 = não, 1 = sim")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

print("=== Bloco de Análise Exploratória ===\n")

features_cont = [col for col in features_cont if col != "age"]

# 1. Histogramas das variáveis contínuas por classe
print("\n=== 1. Histogramas das variáveis contínuas por classe ===")
for feature in features_cont:
    plt.figure(figsize=(8, 4))
    sns.histplot(data=df_limpo, x=feature, hue='cardio', kde=True, bins=30, palette='Set1', element='step')
    plt.title(f'Distribuição de {feature} por Classe (cardio)')
    plt.xlabel(feature)
    plt.ylabel('Frequência')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 2. Countplot de Faixa Etária por Classe
print("\n=== 2. Distribuição de Faixa Etária por Classe ===")
df_limpo["faixa_etaria"] = pd.cut(
    df_limpo["age_years"],
    bins=[20, 30, 40, 50, 60, 70],
    labels=["20-30", "31-40", "41-50", "51-60", "61-70"]
)

plt.figure(figsize=(6, 4))
sns.countplot(data=df_limpo, x='faixa_etaria', hue='cardio', palette='Set1')
plt.title('Distribuição por Faixa Etária e Presença de Doença Cardíaca')
plt.xlabel('Faixa Etária')
plt.ylabel('Contagem')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Countplot de ATIVO (active)
print("\n=== 3. Nível de Atividade Física por Classe ===")
plt.figure(figsize=(6, 4))
sns.countplot(data=df_limpo, x='active', hue='cardio', palette='Set1')
plt.title('Atividade Física vs Doença Cardíaca')
plt.xlabel('Ativo (0 = Não, 1 = Sim)')
plt.ylabel('Contagem')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 4. Countplot de FUMANTE (smoke)
print("\n=== 4. Fumante por Classe ===")
plt.figure(figsize=(6, 4))
sns.countplot(data=df_limpo, x='smoke', hue='cardio', palette='Set1')
plt.title('Fumante vs Doença Cardíaca')
plt.xlabel('Fumante (0 = Não, 1 = Sim)')
plt.ylabel('Contagem')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 5. Pairplot entre variáveis contínuas
print("\n=== 5. Dispersão entre variáveis contínuas (pairplot) ===")
sns.pairplot(
    data=df_limpo,
    vars=features_cont,
    hue='cardio',
    palette='Set1',
    diag_kind='kde',
    plot_kws={'alpha': 0.6}
)
plt.suptitle("Pairplot das Variáveis Contínuas por Classe (cardio)", y=1.02)
plt.show()

# 6. PCA com variáveis CONTÍNUAS (com normalização)
print("\n=== 6. PCA com Variáveis CONTÍNUAS (normalizadas) ===")
scaler_cont = StandardScaler()
X_cont_scaled = scaler_cont.fit_transform(df_limpo[features_cont])

pca_cont = PCA(n_components=2)
X_pca_cont = pca_cont.fit_transform(X_cont_scaled)

plt.figure(figsize=(7, 5))
sns.scatterplot(x=X_pca_cont[:, 0], y=X_pca_cont[:, 1], hue=y, palette='Set1', alpha=0.7)
plt.title("PCA - Variáveis CONTÍNUAS (normalizadas)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 7. PCA com variáveis CONTÍNUAS (normalizadas) + BINÁRIAS/CATEGÓRICAS (sem normalizar)
print("\n=== 7. PCA com Variáveis CONTÍNUAS (normalizadas) + BINÁRIAS/CATEGÓRICAS ===")

scaler_cont = StandardScaler()
X_cont_scaled = scaler_cont.fit_transform(df_limpo[features_cont])
X_cont_df = pd.DataFrame(X_cont_scaled, columns=features_cont, index=df_limpo.index)

X_combined = pd.concat([X_cont_df, df_limpo[features_bin].astype(float)], axis=1)

pca_combined = PCA(n_components=2)
X_pca_combined = pca_combined.fit_transform(X_combined)

plt.figure(figsize=(7, 5))
sns.scatterplot(x=X_pca_combined[:, 0], y=X_pca_combined[:, 1], hue=y, palette='Set1', alpha=0.7)
plt.title("PCA - CONTÍNUAS (normalizadas) + BINÁRIAS/CATEGÓRICAS (sem normalizar)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Normalização com StandardScaler
scaler = StandardScaler()
X_scaled = df_limpo.copy()
X_scaled[features_cont] = scaler.fit_transform(X_scaled[features_cont])

print("=== Histogramas Antes e Depois da Normalização ===")

# Plot comparativo
fig, axes = plt.subplots(len(features_cont), 2, figsize=(12, len(features_cont)*2.5))
fig.suptitle('Comparação de Distribuições: Antes vs. Depois da Normalização', fontsize=16)

for i, feature in enumerate(features_cont):
    axes[i, 0].hist(df_limpo[feature], bins=30, color='blue', alpha=0.7)
    axes[i, 0].set_title(f'Original: {feature}')

    axes[i, 1].hist(X_scaled[feature], bins=30, color='green', alpha=0.7)
    axes[i, 1].set_title(f'Normalizado: {feature}')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

# 1. Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.20, stratify=y, random_state=42)

# 2. Modelo-base
mlp_base = MLPClassifier(
    solver='adam',
    early_stopping=True,
    validation_fraction=0.10,
    random_state=42,
    verbose=False
)

# 3. Grid de hiperparâmetros
param_grid = {
    "hidden_layer_sizes": [(20,), (50,), (100,), (20,20), (50, 50), (100, 100)],
    "activation": ["relu", "tanh", "logistic"],
    "learning_rate_init": [0.001, 0.01, 0.1],
    "max_iter": [500, 750, 1000],
}

# 4. GridSearchCV
grid = GridSearchCV(
    estimator=mlp_base,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1,
    verbose=2
)

# 5. Treinamento
grid.fit(X_train, y_train)

# 6. Avaliação e melhores parâmetros
print("\nMelhores Hiperparâmetros encontrados pelo GridSearchCV:")
best_params = grid.best_params_
for param, valor in best_params.items():
    print(f"• {param}: {valor}")

print(f"\nMelhor acurácia média durante a validação cruzada: {grid.best_score_:.4f}")

# 7. Interpretação
print("\nInterpretação dos parâmetros escolhidos:")
if 'hidden_layer_sizes' in best_params:
    print(f"• hidden_layer_sizes = {best_params['hidden_layer_sizes']} → Define a estrutura da rede. "
          f"Camadas como {best_params['hidden_layer_sizes']} indicam capacidade de capturar padrões mais complexos.")

if 'activation' in best_params:
    ativ = best_params['activation']
    print(f"• activation = '{ativ}' → "
          + ("'relu': Rápido e eficaz para não-linearidades." if ativ == "relu" else
             "'tanh': Bom para dados centrados em 0." if ativ == "tanh" else
             "'logistic': Simples e tradicional."))

if 'learning_rate_init' in best_params:
    lr = best_params['learning_rate_init']
    print(f"• learning_rate_init = {lr} → Controla a velocidade de aprendizagem. Valor intermediário como {lr} busca estabilidade.")

if 'max_iter' in best_params:
    print(f"• max_iter = {best_params['max_iter']} → Número máximo de épocas. Com early stopping, ajuda a evitar overfitting.")

# 8. Top 5 melhores combinações
print("\nTop 5 combinações de hiperparâmetros por acurácia média:")
resultados = pd.DataFrame(grid.cv_results_)
top5 = resultados[['mean_test_score', 'params']].sort_values(by='mean_test_score', ascending=False).head(5)

for idx, row in enumerate(top5.itertuples(), start=1):
    print(f"\n#{idx} - Acurácia média: {row.mean_test_score:.4f}")
    for key, val in row.params.items():
        print(f"   → {key}: {val}")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

# === Avaliação Final do Melhor Modelo (MLP do GridSearch) ===
print("\nAvaliação Final - MLP (Melhor Modelo do GridSearch)\n")

# 1. Obter o melhor estimador
best_mlp = grid.best_estimator_

# 2. Previsões no conjunto de teste (20%)
y_pred_mlp = best_mlp.predict(X_test)

# 3. Relatório de Classificação
print("Classification Report (conjunto de teste):")
print(classification_report(y_test, y_pred_mlp))

# 4. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_mlp)
print("\nMatriz de Confusão:")
print(cm)

# 5. Métricas globais
acc = accuracy_score(y_test, y_pred_mlp)
f1 = f1_score(y_test, y_pred_mlp, average='macro')
print("\nMétricas Globais:")
print(f"• Acurácia : {acc:.4f}")
print(f"• F1-Score : {f1:.4f}")

# 6. Heatmap da Matriz de Confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges')
plt.title('Matriz de Confusão - MLP (Teste)')
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.tight_layout()
plt.show()

# 7. Validação Cruzada no conjunto COMPLETO (não recomendada para avaliação final)
print("\nValidação Cruzada em TODOS os dados (X e y) [apenas exploratória]:")
cv_scores_all = cross_val_score(best_mlp, X, y, cv=5, scoring='accuracy')
print("• Acurácias em cada fold:", cv_scores_all)
print(f"• Acurácia Média (X completo): {cv_scores_all.mean():.4f}")

# 8. Curva de aprendizado (se disponível)
if hasattr(best_mlp, "loss_curve_"):
    print("\nCurva de Aprendizado (Loss por época):")
    plt.figure()
    plt.plot(best_mlp.loss_curve_)
    plt.title("Curva de Aprendizado - MLP")
    plt.xlabel("Épocas")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Modelo não possui atributo 'loss_curve_'.")

# 9. Validação Cruzada nos 20% de TESTE
print("\nValidação Cruzada (cv=5) no CONJUNTO DE TESTE (20%):")
cv_scores_test = cross_val_score(best_mlp, X_test, y_test, cv=5, scoring='accuracy')
print("• Acurácias em cada fold (teste):", cv_scores_test)
print(f"• Acurácia Média (teste): {cv_scores_test.mean():.4f}")

# # Gráfico de Acurácia por Época
# plt.figure()
# plt.plot(historico_acuracia)
# plt.xlabel("Época")
# plt.ylabel("Acurácia")
# plt.title("Acurácia por época")
# plt.grid(True)
# plt.savefig("grafico_acuracia_mlp.png")
# plt.show()
#
# # Gráfico 3D da Perda por Batch e Época
# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")
#
# max_batches = max(len(l) for l in matriz_perda_batches)
# X, Y = np.meshgrid(
#     np.arange(len(matriz_perda_batches)),
#     np.arange(max_batches),
# )
# Z = np.zeros_like(X, dtype=float)
#
# for e, losses in enumerate(matriz_perda_batches):
#     Z[: len(losses), e] = losses
#
# ax.plot_surface(X, Y, Z, cmap="viridis")
# ax.set_xlabel("Época")
# ax.set_ylabel("Batch")
# ax.set_zlabel("Loss")
# ax.set_title("Loss por batch ao longo das épocas")
# plt.savefig("grafico_mapa3d_mlp.png")
# plt.show()